In [ ]:
import pandas as pd
import re
import os
import requests
import http.client
import json
from google.cloud import bigquery
from datetime import datetime
import calendar

# Configuración para acceder a BigQuery
client = bigquery.Client()
# Define el proyecto y el esquema de datos
project_id = "idproyecto"#define el id de tu proyecto de GCP
dataset_id = "datos_abiertos"#define el data set de bigquery con los esquemas de datos
table_id_metadatos = "TBL_metadatos_datos_abiertos" # define el nombre de la tabla de metadatos. Esta debe ir renovándose según appends y según la cantidad de nuevos DF incorporados
#today = datetime.today()
#Esta parte la agregué para parametrizar fechas.
# Calcular el mes y año anterior al actual si es necesario dejar la ejecución manual.
#first_day_previous_month = today.replace(day=1) - timedelta(days=1)
#year = first_day_previous_month.year
#month = first_day_previous_month.month
#nombre_mes = calendar.month_name[month].capitalize()
# Definir el periodo de los datos
year = 2024
month = 10  # Octubre

# Obtener el nombre del mes en español para hacerlo más comprensible para el usuario en los metadatos
nombre_mes = calendar.month_name[month].capitalize()

# Formatear el campo 'period' como "mes año". Se hace esto para incluirlo de forma más claro en los parámetros de los metadatos.
periodo = f"{nombre_mes} {year}"

# Query para obtener todas las filas de la tabla de metadatos y así poder luego inyectar los metadatos en el portal
query_metadatos = f"""
    SELECT name, title, notes, author, author_email, license_id, owner_org, version, private, frequency
    FROM `{project_id}.{dataset_id}.{table_id_metadatos}`
"""

# Ejecutar la consulta de metadatos
job_metadatos = client.query(query_metadatos)
metadatos_df = job_metadatos.result().to_dataframe()

# Iterar sobre cada Fila en la tabla de metadatos y eso implica que cada Fila representa una tabla diferente dentro de tu dataset.
#Es decir, si tienes dos filas de metadatos, deberías tener dos tablas en tu dataset de BQ
for index, row in metadatos_df.iterrows():
    # Extraer los metadatos para el dataset actual
    metadatos = row.to_dict()
    dataset_name = metadatos['name']  # El campo 'name' también es el nombre de la tabla de datos

    # Definir las fechas para la query (primer y último día del mes anterior pensando que quieres que sea un proceso mensual con un mes de desfase)
    first_day_month = datetime(year, month, 1)
    last_day_month = datetime(year, month, calendar.monthrange(year, month)[1])

    # Query para obtener los datos asociados a este dataset (en mi caso debo transformar el campo de fecha pero dependiendo del DF puede ser innecesario
    query_datos = f"""
    SELECT *
    FROM `{project_id}.{dataset_id}.{dataset_name}`
    WHERE DATE(transaction_date) BETWEEN '{first_day_month.strftime('%Y-%m-%d')}' AND '{last_day_month.strftime('%Y-%m-%d')}'
    """

    # Ejecutar la consulta de datos extrayendo el DF según la fecha especificada.
    job_datos = client.query(query_datos)
    datos_df = job_datos.result().to_dataframe()

    # Guardar los datos obtenidos en un archivo CSV con un nombre único. Acá igual puedes jugar con el formato y subirlo en otro formato permitido por Datos Abiertos
    ruta = f'/content/{dataset_name}_{nombre_mes.lower()}_{year}_data.csv'
    datos_df.to_csv(ruta, index=False)

    # Configuración para subir metadatos a la API de datos abiertos
    conn = http.client.HTTPSConnection("datos.gob.cl")
    vartoken ='ingresa tu token'
    headers = {
        'Authorization': vartoken,
        'Content-Type': 'application/json'
    }

    # Usar los metadatos obtenidos de la tabla de BigQuery para construir el payload
    payload = {
        "name": re.sub(r'[^a-z0-9]', '', metadatos['name'].lower()),  # 'name' del payload es el valor del campo 'name' de metadatos
        "title": metadatos['title'],
        "notes": metadatos['notes'],
        "author": metadatos['author'],
        "author_email": metadatos['author_email'],
        "license_id": metadatos['license_id'],
        "owner_org": metadatos['owner_org'],
        "period": periodo,  # El periodo dinámico
        "version": metadatos['version'],
        "private": metadatos['private'],
        "frequency": metadatos['frequency']
    }

    # Convertir el payload a formato JSON
    json_payload = json.dumps(payload)

    # Hacer la solicitud para crear el dataset en la plataforma de datos
    conn.request("POST", "/api/action/package_create", json_payload, headers)
    response = conn.getresponse()
    data = response.read()

    # Imprimir la respuesta de la creación del dataset
    print(f"Dataset {metadatos['name']} creado: {data.decode('utf-8')}")
    print(response.status, response.reason)

    # Ahora vamos a subir el archivo generado a la API
    url = 'https://datos.gob.cl/api/3/action/resource_create'
    data = {
        "package_id": re.sub(r'[^a-z0-9]', '', metadatos['name'].lower()),  # El package_id es el valor del campo 'name' de metadatos
        "description": f"Datos de {periodo}",
        "name": f"{dataset_name}_{nombre_mes.lower()}_{year}_data.csv",
    }
    headers = {
        "Authorization": vartoken,
        "Content-Type": "multipart/form-data"
    }

    # Subir el archivo
    with open(ruta, 'rb') as file:
        files = {'upload': file}
        response = requests.post(url, data=data, headers=headers, files=files)

    # Mostrar el estado de la respuesta de la subida del archivo
    print(f"Status Code: {response.status_code}")
    print(f"Reason: {response.reason}")
    print(f"Response Text: {response.text}")